In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install hazm

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hazm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import os
import re
import json
import copy
import collections

In [10]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# df=pd.read_csv("/content/drive/MyDrive/Digikala-ProductCategorization/cleaned_dataset.csv")
df=pd.read_csv("cleaned_dataset.csv")
df.head(3)

,product_id,product_name,has_image,category1,category2,category3
0,dkp-5747199,جاکارتی طرح هاوایی کد,False,مد و پوشاک,زنانه و مردانه,اکسسوری زنانه و مردانه
1,dkp-2890044,شورت زنانه کد مجموعه عددی,False,مد و پوشاک,زنانه,لباس زنانه
2,dkp-5999158,استند لوازم اداری رومیزی مدل مجموعه عددی,False,کتاب، لوازم تحریر و هنر,لوازم تحریر,لوازم اداری


In [13]:
df =df[df['product_name'].notna()]
print(df.isna().sum(), '\n')

product_id      0
product_name    0
has_image       0
category1       0
category2       0
category3       0
dtype: int64 



In [14]:
df['text_len_by_words'] = df['product_name'].apply(lambda t: len(hazm.word_tokenize(t)))
df.head(3)

,product_id,product_name,has_image,category1,category2,category3,text_len_by_words
0,dkp-5747199,جاکارتی طرح هاوایی کد,False,مد و پوشاک,زنانه و مردانه,اکسسوری زنانه و مردانه,4
1,dkp-2890044,شورت زنانه کد مجموعه عددی,False,مد و پوشاک,زنانه,لباس زنانه,5
2,dkp-5999158,استند لوازم اداری رومیزی مدل مجموعه عددی,False,کتاب، لوازم تحریر و هنر,لوازم تحریر,لوازم اداری,7


In [15]:
def get_name_count(cat_name):
    name, count = np.unique(df[cat_name], return_index=False, return_inverse=False, return_counts=True, axis=None)
    count_sort_ind = np.argsort(count)
    cat3_name=name[count_sort_ind]
    cat3_count=count[count_sort_ind]
    for i in range(len(cat3_name)):
        print(cat3_name[i]+" : "+str(cat3_count[i]))
    return cat3_name,cat3_count

cat2_name,cat2_count=get_name_count("category2")
max_idx=0
for i,cnt in enumerate(cat2_count):
    if cnt<=10:
        max_idx=i

for i in range(max_idx+1):
    df = df.drop(df[df['category2'] == cat2_name[i]].index)
print(df.shape)


cat3_name,cat3_count=get_name_count("category3")
max_idx=0
for i,cnt in enumerate(cat3_count):
    if cnt<=5:
        max_idx=i

for i in range(max_idx+1):
    df = df.drop(df[df['category3'] == cat3_name[i]].index)
print(df.shape)

سرویس‌های اینترنت و سیم‌کارت : 1
آشپرخانه صنعتی و کافی شاپ : 2
ساعت : 2
فینگر فود و پذیرایی : 6
قنادی : 7
کفش ورزشی : 13
بازی و سرگرمی های بومی محلی : 14
تجهیزات ایمنی موتورسیکلت : 30
زیورآلات : 33
مترجم و کتاب صوتی : 39
فرآورده‌های منجمد و یخچالی : 39
لوازم مصرفی موتور سیکلت : 44
مادر و کودک : 51
شور و ترشیجات : 65
میوه و سبزیجات : 76
لبنیات : 80
تبلت و کتابخوان : 83
ابزار دقیق و اندازه گیری : 88
مواد پروتئینی و تخم مرغ : 92
کنسرو و غذای آماده : 100
ایمنی و مراقبت : 108
پوشاک بومی و محلی : 145
گل طبیعی : 147
تجهیزات صنعتی و انبارداری : 186
لوازم یدکی موتور سیکلت : 198
موسیقی : 211
محصولات مذهبی : 216
صبحانه : 218
مکمل غذایی و دارویی : 246
ساعت و مچ بند هوشمند : 246
خانه و کاشانه : 250
محتوای آموزشی : 277
بهداشت شخصی : 281
فیلم : 284
لوازم جانبی موتور سیکلت : 291
نوشیدنی‌های سرد : 296
ابزار دقیق و  اندازه گیری : 360
مواد غذایی ارگانیک : 383
پستانک و لوازم شیردهی : 386
خشکبار و شیرینی : 404
نوشیدنی‌های گرم : 419
گردش و سفر : 430
کالاهای اساسی و خوار و بار : 444
نرم افزار و بازی : 454
تغ

In [16]:
def get_cat_idx(cat,cat_options):
    res=0
    for i,opt in enumerate(cat_options):
        if cat==opt:
            res=i
            break
    return res

In [17]:
category1_options=df["category1"].unique()
category2_options=df["category2"].unique()
category3_options=df["category3"].unique()

In [31]:
def remove_extra_white_speces(s):
    return ' '.join(word for word in s.split() if word)
print(remove_extra_white_speces(category2_options[95]))
cat2_opts=[remove_extra_white_speces(word) for word in category2_options]
cat2_opts=list(set(cat2_opts))
cat3_opts=[remove_extra_white_speces(word) for word in category3_options]
cat3_opts=list(set(cat3_opts))

ابزار دقیق و اندازه گیری


In [34]:
print(len(category3_options))
print(len(cat3_opts))

575
574


In [28]:
print(len(category2_options))
print(category2_options[60])
print(remove_extra_white_speces(category2_options[95]))


96
ابزار دقیق و اندازه گیری
ابزار دقیق و اندازه گیری


In [11]:
df["cat1_id"]=df['category1'].apply(lambda t: get_cat_idx(t,category1_options))
df["cat2_id"]=df['category2'].apply(lambda t: get_cat_idx(t,category2_options))
df["cat3_id"]=df['category3'].apply(lambda t: get_cat_idx(t,category3_options))
df.head(3)

,product_id,product_name,has_image,category1,category2,category3,text_len_by_words,cat1_id,cat2_id,cat3_id
0,dkp-5747199,جاکارتی طرح هاوایی کد,False,مد و پوشاک,زنانه و مردانه,اکسسوری زنانه و مردانه,4,0,0,0
1,dkp-2890044,شورت زنانه کد مجموعه عددی,False,مد و پوشاک,زنانه,لباس زنانه,5,0,1,1
2,dkp-5999158,استند لوازم اداری رومیزی مدل مجموعه عددی,False,کتاب، لوازم تحریر و هنر,لوازم تحریر,لوازم اداری,7,1,2,2


In [13]:
print(len(category1_options))
print(len(category2_options))
print(len(category3_options))

11
96
575


In [14]:
train, test = train_test_split(df, test_size=0.1, random_state=42, stratify=df['cat1_id'])
train, valid = train_test_split(train, test_size=0.1, random_state=42, stratify=train['cat1_id'])
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

# x_train, y_train = train['product_name'].values.tolist(), train['cats'].values.tolist()
# x_valid, y_valid = valid['product_name'].values.tolist(), valid['cats'].values.tolist()
# x_test, y_test = test['product_name'].values.tolist(), test['cats'].values.tolist()


print(train.shape)
print(valid.shape)
print(test.shape)

(428503, 10)
(47612, 10)
(52902, 10)


In [15]:
class DigikalaDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Digikala. """

    def __init__(self, tokenizer, product_names, targets1,targets2,targets3, max_len=128):
        self.product_names = product_names
        self.targets1 = targets1
        self.targets2=targets2
        self.targets3=targets3
        self.has_target = isinstance(targets1, list) or isinstance(targets1, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len


        # self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}

    def __len__(self):
        return len(self.product_names)

    def __getitem__(self, item):
        product_name = str(self.product_names[item])

        if self.has_target:
            target1=self.targets1[item]
            target2=self.targets2[item]
            target3=self.targets3[item]

            # target = self.label_map.get(str(self.targets[item]), str(self.targets[item]))

        encoding = self.tokenizer.encode_plus(
            product_name,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'product_name': product_name,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets1'] = torch.tensor(target1, dtype=torch.long)
            inputs['targets2'] = torch.tensor(target2, dtype=torch.long)
            inputs['targets3'] = torch.tensor(target3, dtype=torch.long)

        return inputs


def create_data_loader(x, t1,t2,t3, tokenizer, max_len, batch_size):
    dataset = DigikalaDataset(
        product_names=x,
        targets1=t1,
        targets2=t2,
        targets3=t3,
        tokenizer=tokenizer,
        max_len=max_len)

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=False)

In [16]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16
# setup the tokenizer and configuration
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

In [17]:
train_data_loader = create_data_loader(train['product_name'].to_numpy(), train['cat1_id'].to_numpy(),train['cat2_id'].to_numpy(),train['cat3_id'].to_numpy(), tokenizer, max_len=MAX_LEN, batch_size=TRAIN_BATCH_SIZE)
valid_data_loader = create_data_loader(valid['product_name'].to_numpy(), valid['cat1_id'].to_numpy(),valid['cat2_id'].to_numpy(),valid['cat3_id'].to_numpy(),tokenizer,max_len= MAX_LEN, batch_size=VALID_BATCH_SIZE)
test_data_loader = create_data_loader(test['product_name'].to_numpy(), test['cat1_id'].to_numpy(),test['cat2_id'].to_numpy(),test['cat3_id'].to_numpy(), tokenizer, max_len=MAX_LEN, batch_size=TEST_BATCH_SIZE)
# test_data_loader = create_data_loader(test['product_name'].to_numpy()[:500], test['cat1_id'].to_numpy()[:500],test['cat2_id'].to_numpy()[:500],test['cat3_id'].to_numpy()[:500], tokenizer, max_len=MAX_LEN, batch_size=TEST_BATCH_SIZE)

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [20]:
OUTPUT_PATH="/content/drive/MyDrive/Digikala-ProductCategorization/models/"

## Level1

In [21]:
# create a key finder based on label 2 id and id to label
LEVEL1_MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'

level1_label2id = {label: i for i, label in enumerate(category1_options)}
level1_id2label = {v: k for k, v in level1_label2id.items()}

print(f'label2id: {level1_label2id}')
print(f'id2label: {level1_id2label}')
# setup the tokenizer and configuration

level1_tokenizer = BertTokenizer.from_pretrained(LEVEL1_MODEL_NAME_OR_PATH)
level1_config = BertConfig.from_pretrained(
    LEVEL1_MODEL_NAME_OR_PATH, **{
        'label2id': level1_label2id,
        'id2label': level1_id2label,
    })

print(level1_config.to_json_string())

label2id: {'مد و پوشاک': 0, 'کتاب، لوازم تحریر و هنر': 1, 'کالاهای سوپرمارکتی': 2, 'کالای دیجیتال': 3, 'خانه و آشپزخانه': 4, 'خودرو و موتورسیکلت': 5, 'ابزار آلات و تجهیزات': 6, 'ورزش و سفر': 7, 'زیبایی و سلامت': 8, 'اسباب بازی، کودک و نوزاد': 9, 'محصولات بومی و محلی': 10}
id2label: {0: 'مد و پوشاک', 1: 'کتاب، لوازم تحریر و هنر', 2: 'کالاهای سوپرمارکتی', 3: 'کالای دیجیتال', 4: 'خانه و آشپزخانه', 5: 'خودرو و موتورسیکلت', 6: 'ابزار آلات و تجهیزات', 7: 'ورزش و سفر', 8: 'زیبایی و سلامت', 9: 'اسباب بازی، کودک و نوزاد', 10: 'محصولات بومی و محلی'}
{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u0645\u062f \u0648 \u067e\u0648\u0634\u0627\u06a9",
    "1": "\u06a9\u062a\u0627\u0628\u060c \u0644\u0648\u0627\u0632\u0645 \u062a\u062d\u0631\u06cc\u0631 \u0648 \u0647\u0646\u0631",
    "2": "\u06a9\u0627\u0644\u0627\u0647\u0627\u0

In [22]:
level1_config.num_labels

11

In [23]:
class Level1Model(nn.Module):

    def __init__(self, config):
        super(Level1Model, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(bert_output["pooler_output"])
        logits = self.classifier(pooled_output)
        return logits

In [24]:
lvl1_model = Level1Model(config=level1_config)
lvl1_model = lvl1_model.to(device)

print('lvl1_model', type(lvl1_model))

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

lvl1_model <class '__main__.Level1Model'>


In [25]:
# level1_last_epoch=1
level1_checkpoint = torch.load(OUTPUT_PATH+f'checkpoint_ep2_1.pt')
lvl1_model.load_state_dict(level1_checkpoint['model_state_dict'])
level1_step=level1_checkpoint["step"]
level1_eval_loss_min = level1_checkpoint['eval_loss_min']

## Level2

In [26]:
# create a key finder based on label 2 id and id to label
LEVEL2_MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'

level2_label2id = {label: i for i, label in enumerate(category2_options)}
level2_id2label = {v: k for k, v in level2_label2id.items()}

print(f'label2id: {level2_label2id}')
print(f'id2label: {level2_id2label}')
# setup the tokenizer and configuration

level2_tokenizer = BertTokenizer.from_pretrained(LEVEL2_MODEL_NAME_OR_PATH)
level2_config = BertConfig.from_pretrained(
    LEVEL2_MODEL_NAME_OR_PATH, **{
        'label2id': level2_label2id,
        'id2label': level2_id2label,
    })

print(level2_config.to_json_string())

label2id: {'زنانه و مردانه': 0, 'زنانه': 1, 'لوازم تحریر': 2, 'نوشیدنی\u200cهای سرد': 3, 'کامپیوتر و تجهیزات جانبی': 4, 'دکوراتیو': 5, 'لوازم جانبی کالای دیجیتال': 6, 'لوازم مصرفی خودرو': 7, 'کتاب': 8, 'ابزار غیر برقی': 9, 'لوازم سفر و کمپینگ': 10, 'لوازم آرایشی': 11, 'بچگانه': 12, 'خواب کودک': 13, 'تنقلات': 14, 'سرو و پذیرایی': 15, 'لوازم ورزشی': 16, 'مردانه': 17, 'ابزار برقی و شارژی': 18, 'لوازم بهداشتی': 19, 'لپ تاپ': 20, 'باغبانی و کشاورزی': 21, 'طلا و جواهرات': 22, 'یراق آلات': 23, 'سرگرمی و آموزشی': 24, 'حمام و سرویس بهداشتی': 25, 'خواب': 26, 'روشنایی و الکتریکی': 27, 'شستشو و نظافت': 28, 'دوربین': 29, 'لوازم خانگی برقی': 30, 'کنسول بازی': 31, 'گردش و سفر': 32, 'ابزار سلامت': 33, 'صنایع دستی': 34, 'آشپزخانه': 35, 'لوازم یدکی خودرو': 36, 'پت شاپ': 37, 'مکمل غذایی و دارویی': 38, 'ملزومات اتاق کودک': 39, 'لوازم و مصالح ساختمانی': 40, 'صوتی و تصویری': 41, 'فرش و گلیم': 42, 'پوشاک ورزشی': 43, 'لوازم جانبی خودرو': 44, 'پوشاک بومی و محلی': 45, 'ماشین های اداری': 46, 'تجهیزات صنعتی و انب

In [27]:
level2_config.num_labels

96

In [28]:
class Level2Model(nn.Module):

    def __init__(self, config):
        super(Level2Model, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(bert_output["pooler_output"])
        logits = self.classifier(pooled_output)
        return logits

In [29]:
lvl2_model = Level2Model(config=level2_config)
lvl2_model = lvl2_model.to(device)

print('lvl2_model', type(lvl2_model))

lvl2_model <class '__main__.Level2Model'>


In [30]:
level2_checkpoint = torch.load(OUTPUT_PATH+f'level2_checkpoint_ep3_1.pt')
lvl2_model.load_state_dict(level2_checkpoint['model_state_dict'])
level2_step=level2_checkpoint["step"]
level2_eval_loss_min = level2_checkpoint['eval_loss_min']

## Level3

In [31]:
# create a key finder based on label 2 id and id to label
LEVEL3_MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'

level3_label2id = {label: i for i, label in enumerate(category3_options)}
level3_id2label = {v: k for k, v in level3_label2id.items()}

print(f'label2id: {level3_label2id}')
print(f'id2label: {level3_id2label}')
# setup the tokenizer and configuration

level3_tokenizer = BertTokenizer.from_pretrained(LEVEL3_MODEL_NAME_OR_PATH)
level3_config = BertConfig.from_pretrained(
    LEVEL3_MODEL_NAME_OR_PATH, **{
        'label2id': level3_label2id,
        'id2label': level3_id2label,
    })

print(level3_config.to_json_string())

label2id: {'اکسسوری زنانه و مردانه': 0, 'لباس زنانه': 1, 'لوازم اداری': 2, 'سالنامه': 3, 'عرقیات و گلاب': 4, 'هدفون، هدست، میکروفون': 5, 'استیکر و پوستر': 6, 'لوازم جانبی گوشی موبایل': 7, 'روغن و ضد یخ خودرو': 8, 'کتاب هنر و سرگرمی': 9, 'ابزار دستی': 10, 'اکسسوری زنانه': 11, 'ساک و چمدان': 12, 'آرایش چشم': 13, 'کتاب شعر و ادبیات': 14, 'دفتر و کاغذ و مقوا': 15, 'پسرانه': 16, 'سرویس خواب کودک و نوزاد': 17, 'بیسکویت و ویفر': 18, 'پارچ، بطری و لیوان': 19, 'چادر': 20, 'ابزار مهمانی': 21, 'قاب عکس و تابلو': 22, 'ورزش های هوازی و تناسب اندام': 23, 'اکسسوری مردانه': 24, 'قطعات الکترونیکی': 25, 'لباس مردانه': 26, 'دخترانه': 27, 'کفش زنانه': 28, 'مراقبت پوست': 29, 'لوازم تزیینی': 30, 'کفش مردانه': 31, 'لوازم جانبی لپ تاپ': 32, 'ابزارآلات باغبانی': 33, 'زیورآلات طلا زنانه': 34, 'لوازم جانبی ساعت و مچ بند هوشمند': 35, 'دستگیره در': 36, 'اسباب بازی': 37, 'حمام': 38, 'سرویس خواب': 39, 'نور و روشنایی': 40, 'نظافت لباس': 41, 'لوازم جانبی عکاسی و فیلم برداری': 42, 'ملزومات هدیه': 43, 'نوزاد': 44, 'قمقم

In [32]:
level3_config.num_labels

575

In [33]:
class Level3Model(nn.Module):

    def __init__(self, config):
        super(Level3Model, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(bert_output["pooler_output"])
        logits = self.classifier(pooled_output)
        return logits

In [34]:
lvl3_model = Level3Model(config=level3_config)
lvl3_model = lvl3_model.to(device)

print('lvl3_model', type(lvl3_model))

lvl3_model <class '__main__.Level3Model'>


In [35]:
level3_checkpoint = torch.load(OUTPUT_PATH+f'level3_checkpoint_ep2_2.pt')
lvl3_model.load_state_dict(level3_checkpoint['model_state_dict'])
level3_step=level3_checkpoint["step"]
level3_eval_loss_min = level3_checkpoint['eval_loss_min']

## Test

In [36]:
def extract_name(tokenizer,input_ids,index):
  current_ids=input_ids[index]
  tokens=[]
  for i in current_ids:
    if i not in tokenizer.all_special_ids:
      tokens.append(i)
  name=tokenizer.decode(tokens)
  return name


In [37]:
def predict(model, product_names, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(product_names, None,None,None, tokenizer, max_len, batch_size)

    predictions = []
    prediction_probs = []


    model.eval()
    with torch.no_grad():
        for dl in data_loader:
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))


    # predictions = torch.stack(predictions).cpu().detach().numpy()
    # prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [38]:
def testing(level1_model,level2_model,level3_model, data_loader, tokenizer, max_len=128, batch_size=32):
    # data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size)

    first_level_predictions = []
    first_level_true_targets=[]
    first_level_prediction_probs = []

    second_level_predictions = []
    second_level_true_targets=[]
    second_level_prediction_probs = []

    third_level_predictions = []
    third_level_true_targets=[]
    third_level_prediction_probs = []


    level1_model.eval()
    level2_model.eval()
    level3_model.eval()

    # xx=0
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets1 = dl['targets1']
            targets2 = dl['targets2']
            targets3 = dl['targets3']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets1 = targets1.to(device)
            targets2 = targets2.to(device)
            targets3 = targets3.to(device)

            # print("************")
            # print(type(targets1))
            # print(targets1)
            # print(targets1.shape)
            first_level_true_targ=[]
            second_level_true_targ=[]
            third_level_true_targ=[]
            for i in range(targets1.shape[0]):
              first_level_true_targ.append(targets1[i])
              second_level_true_targ.append(targets2[i])
              third_level_true_targ.append(targets3[i])

            # print(first_level_true_targ)
            # print(second_level_true_targ)
            # print(third_level_true_targ)
            # break

            # compute predicted outputs by passing inputs to the model
            outputs = level1_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            first_level_predictions.extend(preds)
            first_level_true_targets.extend(first_level_true_targ)
            first_level_prediction_probs.extend(F.softmax(outputs, dim=1))


            # print(first_level_predictions)
            # print(len(first_level_predictions))
            # print(type(input_ids))
            # print(input_ids.shape)

            product_names=[]
            for i in range(input_ids.shape[0]):
              name=extract_name(tokenizer,input_ids,i)
              product_names.append(name)

            # print(product_names)
            product_names_with_category1=[]
            for i,name in enumerate(product_names):
              name=category1_options[preds[i]]+" -"+name
              product_names_with_category1.append(name)
            # print("Product names with category1")
            # print(product_names_with_category1)

            second_level_preds,second_level_preds_probs=predict(lvl2_model,product_names_with_category1,tokenizer,batch_size=batch_size)
            # print(product_names)
            product_names_with_category2=[]
            for i,name in enumerate(product_names):
              name=category1_options[preds[i]]+" -"+category2_options[second_level_preds[i]]+" -"+name
              product_names_with_category2.append(name)

            # print("Product names with category2")
            # print(product_names_with_category2)


            third_level_preds,third_level_pred_probs=predict(lvl3_model,product_names_with_category2,tokenizer,batch_size=batch_size)


            second_level_predictions.extend(second_level_preds)
            second_level_true_targets.extend(second_level_true_targ)

            third_level_predictions.extend(third_level_preds)
            third_level_true_targets.extend(third_level_true_targ)




    first_cat_pred_index=[]
    first_cat_true_index=[]

    second_cat_pred_index=[]
    second_cat_true_index=[]

    third_cat_pred_index=[]
    third_cat_true_index=[]


    # predictions = torch.stack(predictions).cpu().detach().numpy()
    # prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    predictions=[]
    true_values=[]
    for i in range(len(first_level_predictions)):
      a=str(first_level_predictions[i].item())+str(second_level_predictions[i].item())+str(third_level_predictions[i].item())
      # print(a)
      first_cat_pred_index.append(first_level_predictions[i].item())
      second_cat_pred_index.append(second_level_predictions[i].item())
      third_cat_pred_index.append(third_level_predictions[i].item())

      predictions.append(a)

    for i in range(len(first_level_true_targets)):
      a=str(first_level_true_targets[i].item())+str(second_level_true_targets[i].item())+str(third_level_true_targets[i].item())
      first_cat_true_index.append(first_level_true_targets[i].item())
      second_cat_true_index.append(second_level_true_targets[i].item())
      third_cat_true_index.append(third_level_true_targets[i].item())


      true_values.append(a)

    cat_preds=[first_cat_pred_index,second_cat_pred_index,third_cat_pred_index]
    cat_trues=[first_cat_true_index,second_cat_true_index,third_cat_true_index]


    # first_y_true = torch.stack(first_level_true_targets).cpu().detach().numpy()
    # first_y_pred = torch.stack(first_level_predictions).cpu().detach().numpy()

    # second_y_true = torch.stack(second_level_true_targets).cpu().detach().numpy()
    # second_y_pred = torch.stack(second_level_predictions).cpu().detach().numpy()

    # third_y_true = torch.stack(third_level_true_targets).cpu().detach().numpy()
    # third_y_pred = torch.stack(third_level_predictions).cpu().detach().numpy()


    # y = [y_true, y_pred]
    true_values2=[]
    predictions2=[]
    for val in true_values:
      true_values2.append(torch.tensor(int(val)))

    for val in predictions:
      predictions2.append(torch.tensor(int(val)))


    true_values=true_values2
    predictions=predictions2
    y_true = torch.stack(true_values).cpu().detach().numpy()
    y_pred = torch.stack(predictions).cpu().detach().numpy()

    y=[y_true,y_pred]
    # print(predictions)
    # print(true_values)
    # print(first_level_true_targets)
    # print(third_level_true_targets)
    return y,cat_preds,cat_trues

In [39]:
def single_level_testing(model, data_loader, tokenizer,target_name, max_len=128, batch_size=32):
    # data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size)

    predictions = []
    true_targets=[]
    prediction_probs = []


    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl[target_name]

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)


            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            true_targets.extend(targets)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    # predictions = torch.stack(predictions).cpu().detach().numpy()
    # prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()
    y_true = torch.stack(true_targets).cpu().detach().numpy()
    y_pred = torch.stack(predictions).cpu().detach().numpy()
    y = [y_true, y_pred]

    return y, prediction_probs

In [40]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

In [41]:
test_y,cat_preds,cat_trues = testing(lvl1_model,lvl2_model,lvl3_model, test_data_loader, tokenizer, max_len=128)

  0%|          | 0/3307 [00:00<?, ?it/s]

In [43]:
acc_and_f1(np.array(cat_trues[0]), np.array(cat_preds[0]), average='weighted')

{'acc': 0.9842917091981399, 'f1': 0.9841908945919325}

In [44]:
acc_and_f1(np.array(cat_trues[1]), np.array(cat_preds[1]), average='weighted')

{'acc': 0.9733091376507504, 'f1': 0.9731646233354612}

In [45]:
acc_and_f1(np.array(cat_trues[2]), np.array(cat_preds[2]), average='weighted')

{'acc': 0.9576197497259082, 'f1': 0.9562909899202364}

In [72]:
def save_list(list_to_save,file_name):
  with open(f'{file_name}.txt', 'w') as fp:
    for item in list_to_save:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

In [73]:
save_list(cat_preds[0],"cat_pred_0")
save_list(cat_preds[1],"cat_pred_1")
save_list(cat_preds[2],"cat_pred_2")


save_list(cat_trues[0],"cat_trues_0")
save_list(cat_trues[1],"cat_trues_1")
save_list(cat_trues[2],"cat_trues_2")

Done
Done
Done
Done
Done
Done


In [1]:
def read_list_from_file(file_name):
  list_to_read = []
  # open file and read the content in a list
  with open(f'{file_name}.txt', 'r') as fp:
      for line in fp:
          # remove linebreak from a current name
          # linebreak is the last character of each line
          x = line[:-1]

          # add current item to the list
          list_to_read.append(x)
  return list_to_read

In [3]:
a0=read_list_from_file("cat_pred_0")
a0=[int(i) for i in a0]
# print(a0==cat_preds[0])

a1=read_list_from_file("cat_pred_1")
a1=[int(i) for i in a1]
# print(a1==cat_preds[1])


a2=read_list_from_file("cat_pred_2")
a2=[int(i) for i in a2]
# print(a2==cat_preds[2])



a0=read_list_from_file("cat_trues_0")
a0=[int(i) for i in a0]
# print(a0==cat_trues[0])

a1=read_list_from_file("cat_trues_1")
a1=[int(i) for i in a1]
# print(a1==cat_trues[1])


a2=read_list_from_file("cat_trues_2")
a2=[int(i) for i in a2]
# print(a2==cat_trues[2])


In [4]:
cat_preds=[]
cat_trues=[]


a0=read_list_from_file("cat_pred_0")
cat_preds.append(a0)
a0=read_list_from_file("cat_pred_1")
cat_preds.append(a0)
a0=read_list_from_file("cat_pred_2")
cat_preds.append(a0)


a0=read_list_from_file("cat_trues_0")
cat_trues.append(a0)

a0=read_list_from_file("cat_trues_1")
cat_trues.append(a0)

a0=read_list_from_file("cat_trues_2")
cat_trues.append(a0)

In [5]:
first_cat_falses=[]
for i in range(len(cat_preds[0])):
  if cat_preds[0][i]!=cat_trues[0][i]:
    first_cat_falses.append(i)

second_cat_falses=[]
for i in range(len(cat_preds[1])):
  if cat_preds[1][i]!=cat_trues[1][i]:
    second_cat_falses.append(i)

third_cat_falses=[]
for i in range(len(cat_preds[2])):
  if cat_preds[2][i]!=cat_trues[2][i]:
    third_cat_falses.append(i)

In [6]:
print(len(first_cat_falses))
print(len(second_cat_falses))
print(len(third_cat_falses))

second_true_first_false=[]
for item in first_cat_falses:
  if item not in second_cat_falses:
    second_true_first_false.append(item)

third_ture_second_false=[]
for item in second_cat_falses:
  if item not in third_cat_falses:
    third_ture_second_false.append(item)


third_ture_first_false=[]
for item in first_cat_falses:
  if item not in third_cat_falses:
    third_ture_first_false.append(item)

831
1412
2242


In [7]:
print(len(second_true_first_false))
print(len(third_ture_second_false))
print(len(third_ture_first_false))


2
13
2


In [8]:
print(second_true_first_false)
print(third_ture_second_false)
print(third_ture_first_false)

[2348, 25879]
[155, 6602, 11694, 12917, 16816, 20469, 20800, 24242, 24582, 32888, 37396, 45429, 50368]
[2348, 25879]


In [66]:
for i in second_true_first_false:
  name= test['product_name'].to_numpy()[i]
  products=[name]
  pred,_=predict_users_inputs(lvl1_model,products,tokenizer,max_len=128,batch_size=len(products))
  print("-"*80)
  print(name)
  print(f"level1 predictions: {category1_options[pred]}, label id: {pred}")
  print(f"level1 True label: {category1_options[[test['cat1_id'].to_numpy()[i]]]},  label id: {[test['cat1_id'].to_numpy()[i]]}")
  # print(pred)
  lvl2_products=[]
  for j in range(len(products)):
    category1_idx=pred[j]
    # print(products[j])
    lvl2_products.append(category1_options[category1_idx]+" -"+products[j])
    level2_predictions,_=predict_users_inputs(lvl2_model,lvl2_products,tokenizer,max_len=128,batch_size=len(lvl2_products))
    print(f"level2 predictions: {category2_options[level2_predictions]}, label id : {level2_predictions}")
    print(f"level2 True label:{category2_options[[test['cat2_id'].to_numpy()[i]]]} label id: {[test['cat2_id'].to_numpy()[i]]}")
  # break



  0%|          | 0/1 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
اسپری ضد عفونی کننده حیوانات خانگی کروپت مدل   حجم  میلی‌لیتر
level1 predictions: ['زیبایی و سلامت'], label id: [8]
level1 True label: ['خانه و آشپزخانه'],  label id: [4]


  0%|          | 0/1 [00:00<?, ?it/s]

level2 predictions: ['پت شاپ'], label id : [37]
level2 True label:['پت شاپ'] label id: [37]


  0%|          | 0/1 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
مایع تمیز کننده دستگاه قهوه ساز ملیتا مدل   حجم  میلی‌لیتر
level1 predictions: ['کالاهای سوپرمارکتی'], label id: [2]
level1 True label: ['خانه و آشپزخانه'],  label id: [4]


  0%|          | 0/1 [00:00<?, ?it/s]

level2 predictions: ['لوازم خانگی برقی'], label id : [30]
level2 True label:['لوازم خانگی برقی'] label id: [30]


In [53]:
def predict_users_inputs(model, product_names, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(product_names, None,None,None, tokenizer, max_len, batch_size)

    predictions = []
    prediction_probs = []


    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [50]:
from sklearn.metrics import classification_report
test_score = acc_and_f1(test_y[0], test_y[1], average='weighted')

In [51]:
test_score

{'acc': 0.95733620657064, 'f1': 0.9561095936651491}

In [ ]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)


['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 4, 0, 2, 3]


## Predict

In [ ]:
product="تیشرت مردانه"
product2="ساعت مچی عقربه‌ای زنانه "
product3="شامپو بدن مردانه"
product4="قاب گوشی اندروید"
product5="شیر پرچرب"
product6="گوشی موبایل مدل اندروید"
product7="هندزفری بلوتوثی"
product8="ساعت هوشمند اپل واچ"
products=[product,product2,product3,product4,product5,product6,product7,product8]


In [ ]:
lvl1_predictions,lvl1_prediction_probs=predict_users_inputs(lvl1_model,products,tokenizer,max_len=128,batch_size=len(products))

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
lvl2_products=[]
for i in range(len(products)):
  category1_idx=lvl1_predictions[i]
  print(products[i])
  print(category1_options[category1_idx])
  lvl2_products.append(category1_options[category1_idx]+" -"+products[i])
  print("-"*80)

تیشرت مردانه
مد و پوشاک
--------------------------------------------------------------------------------
ساعت مچی عقربه‌ای زنانه 
مد و پوشاک
--------------------------------------------------------------------------------
شامپو بدن مردانه
زیبایی و سلامت
--------------------------------------------------------------------------------
قاب گوشی اندروید
کالای دیجیتال
--------------------------------------------------------------------------------
شیر پرچرب
کالاهای سوپرمارکتی
--------------------------------------------------------------------------------
گوشی موبایل مدل اندروید
کالای دیجیتال
--------------------------------------------------------------------------------
هندزفری بلوتوثی
کالای دیجیتال
--------------------------------------------------------------------------------
ساعت هوشمند اپل واچ
کالای دیجیتال
--------------------------------------------------------------------------------


In [ ]:
level2_predictions,level2_prediction_probs=predict_users_inputs(lvl2_model,lvl2_products,tokenizer,max_len=128,batch_size=len(lvl2_products))

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
lvl3_products=[]
for i in range(len(products)):
  category1_idx=lvl1_predictions[i]
  category2_idx=level2_predictions[i]
  print(products[i])
  print(category1_options[category1_idx])
  print(category2_options[category2_idx])
  lvl3_products.append(category1_options[category1_idx]+" -"+category2_options[category2_idx]+" -"+products[i])
  print("-"*80)

تیشرت مردانه
مد و پوشاک
مردانه
--------------------------------------------------------------------------------
ساعت مچی عقربه‌ای زنانه 
مد و پوشاک
زنانه
--------------------------------------------------------------------------------
شامپو بدن مردانه
زیبایی و سلامت
لوازم بهداشتی
--------------------------------------------------------------------------------
قاب گوشی اندروید
کالای دیجیتال
لوازم جانبی کالای دیجیتال
--------------------------------------------------------------------------------
شیر پرچرب
کالاهای سوپرمارکتی
لبنیات
--------------------------------------------------------------------------------
گوشی موبایل مدل اندروید
کالای دیجیتال
لوازم جانبی کالای دیجیتال
--------------------------------------------------------------------------------
هندزفری بلوتوثی
کالای دیجیتال
کامپیوتر و تجهیزات جانبی
--------------------------------------------------------------------------------
ساعت هوشمند اپل واچ
کالای دیجیتال
ساعت و مچ بند هوشمند
-----------------------------------------------

In [ ]:
level3_predictions,level3_prediction_probs=predict_users_inputs(lvl3_model,lvl3_products,tokenizer,max_len=128,batch_size=len(lvl3_products))

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
for i in range(len(products)):

  category1_idx=lvl1_predictions[i]
  category2_idx=level2_predictions[i]
  category3_idx=level3_predictions[i]
  print(products[i])
  print(category1_options[category1_idx]+" -"+category2_options[category2_idx]+" -"+category3_options[category3_idx])
  print("-"*80)

تیشرت مردانه
مد و پوشاک -مردانه -لباس مردانه
--------------------------------------------------------------------------------
ساعت مچی عقربه‌ای زنانه 
مد و پوشاک -زنانه -اکسسوری زنانه
--------------------------------------------------------------------------------
شامپو بدن مردانه
زیبایی و سلامت -لوازم بهداشتی -بهداشت و مراقبت بدن
--------------------------------------------------------------------------------
قاب گوشی اندروید
کالای دیجیتال -لوازم جانبی کالای دیجیتال -لوازم جانبی گوشی موبایل
--------------------------------------------------------------------------------
شیر پرچرب
کالاهای سوپرمارکتی -لبنیات -شیر
--------------------------------------------------------------------------------
گوشی موبایل مدل اندروید
کالای دیجیتال -لوازم جانبی کالای دیجیتال -لوازم جانبی گوشی موبایل
--------------------------------------------------------------------------------
هندزفری بلوتوثی
کالای دیجیتال -کامپیوتر و تجهیزات جانبی -هدفون، هدست، میکروفون
-------------------------------------------------